In [ ]:
import json

In [ ]:
!pip install -U sentence_transformers

In [ ]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('all-MiniLM-L6-v2')

Reading the data form CSV **(DONT RUN)**



In [ ]:
file = open("tools_dictionary.json")
data = json.load(file)

DATA CSV **(RUN THIS INSTEAD)**

In [ ]:
data = {
  "create_actionable_tasks_from_text": {
    "Description": "Given a text, extracts actionable insights, and creates tasks for them, which are kind of a work item.",
    "ArgumentName": {
      "text": {
        "Description": "The text from which the actionable insights need to be created.",
        "ArgumentType": "string"
      }
    }
  },
  "get_similar_work_items": {
    "Description": "Returns a list of work items that are similar to the given work item",
    "ArgumentName": {
      "work_id": {
        "Description": "The ID of the work item for which you want to find similar items",
        "ArgumentType": "string"
      }
    }
  },
  "get_sprint_id": {
    "Description": "Returns the ID of the current sprint",
    "ArgumentName": {
      "_": {
        "Description":"_",
        "ArgumentType":"_"
      }
    }
  },
  "prioritize_objects": {
    "Description": "Returns a list of objects sorted by priority. The logic of what constitutes priority for a given object is an internal implementation detail.",
    "ArgumentName": {
      "objects": {
        "Description": "A list of objects to be prioritized",
        "ArgumentType": "array of objects"
      }
    }
  },
  "search_object_by_name": {
    "Description": "Given a search string, returns the id of a matching object in the system of record. If multiple matches are found, it returns the one where the confidence is highest.",
    "ArgumentName": {
      "query": {
        "Description": "The search string, could be for example customer\u2019s name, part name, user name.",
        "ArgumentType": "string"
      }
    }
  },
  "summarize_objects": {
    "Description": "Summarizes a list of objects. The logic of how to summarize a particular object type is an internal\nimplementation\ndetail.",
    "ArgumentName": {
      "objects": {
        "Description": "List of objects to summarize",
        "ArgumentType": "array of objects"
      }
    }
  },
  "who_am_i": {
    "Description": "Returns the ID of the current user",
    "ArgumentName": {
      "_": {
        "Description":"_",
        "ArgumentType":"_"
      }
    }
  },
  "works_list": {
    "Description": "Returns a list of work items matching the request",
    "ArgumentName": {
      "applies_to_part": {
        "Description": "Filters for work belonging to any of the provided parts",
        "ArgumentType": "array of strings"
      },
      "created_by": {
        "Description": "Filters for work created by any of these users",
        "ArgumentType": "array of strings"
      },
      "issue.priority": {
        "Description": "Filters for issues with any of the provided priorities. Allowed values: p0, p1, p2,  p3",
        "ArgumentType": "array of strings"
      },
      "issue.rev_orgs": {
        "Description": "Filters for issues with any of the provided Rev organizations",
        "ArgumentType": "array of strings"
      },
      "limit": {
        "Description": "The maximum number of works to return. The default is '50'",
        "ArgumentType": "integer (int32)"
      },
      "owned_by": {
        "Description": "Filters for work owned by any of these users",
        "ArgumentType": "array of strings"
      },
      "stage.name": {
        "Description": "Filters for records in the provided stage(s) by name",
        "ArgumentType": "array of strings"
      },
      "ticket.needs_response": {
        "Description": "Filters for tickets that need a response",
        "ArgumentType": "boolean"
      },
      "ticket.rev_org": {
        "Description": "Filters for tickets associated with any of the provided Rev organizations",
        "ArgumentType": "array of strings"
      },
      "ticket.severity": {
        "Description": "Filters for tickets with any of the provided severities. Allowed",
        "ArgumentType": "array of strings"
      },
      "_": {
        "Description": "values: blocker, high, low, medium",
        "ArgumentType":"_"
      },
      "ticket.source_channel": {
        "Description": "Filters for tickets with any of the provided source channels",
        "ArgumentType": "array of strings"
      },
      "type": {
        "Description": "Filters for work of the provided types. Allowed values: issue, ticket, task",
        "ArgumentType": "array of strings"
      }
    }
  },
  "add_work_items_to_sprint": {
    "Description": "Adds the given work items to the sprint",
    "ArgumentName": {
      "work_ids": {
        "Description": "A list of work item IDs to be added to the sprint.",
        "ArgumentType": "array of strings"
      },
      "sprint_id": {
        "Description": "The ID of the sprint to which the work items should be added",
        "ArgumentType": "str"
      }
    }
  }
}

Extracting Argument Descriptions

In [ ]:
tools = []
tool_descriptions = []
arg_names = []
arg_descriptions = []

for key, value in data.items():
    tmp_tool = key
    tools.append(tmp_tool)
    tmp_tool_dsc = value.get("Description")
    tool_descriptions.append(tmp_tool_dsc)
    tmp_arg_dsc = value.get("ArgumentName")
    for argName, argDesc in tmp_arg_dsc.items():
        if (argDesc.get("Description") == "_"):
            arg_names.append(key)
            arg_descriptions.append(tmp_tool_dsc)
        else:
            arg_names.append(argName)
            arg_descriptions.append(argDesc.get("Description"))

arguments = {arg_names[i]: arg_descriptions[i] for i in range(len(arg_names))}

print(*arg_descriptions, sep = "\n")
# print(arguments)

The text from which the actionable insights need to be created.
The ID of the work item for which you want to find similar items
Returns the ID of the current sprint
A list of objects to be prioritized
The search string, could be for example customer’s name, part name, user name.
List of objects to summarize
Returns the ID of the current user
Filters for work belonging to any of the provided parts
Filters for work created by any of these users
Filters for issues with any of the provided priorities. Allowed values: p0, p1, p2,  p3
Filters for issues with any of the provided Rev organizations
The maximum number of works to return. The default is '50'
Filters for work owned by any of these users
Filters for records in the provided stage(s) by name
Filters for tickets that need a response
Filters for tickets associated with any of the provided Rev organizations
Filters for tickets with any of the provided severities. Allowed
values: blocker, high, low, medium
Filters for tickets with any o

Encoding All Desciptions and findng maximum Similarity

In [ ]:
query = "Priority List"
argEncodings = model.encode(arg_descriptions)
queryEncoding = model.encode(query)

CosineSimilarity = util.cos_sim(argEncodings, queryEncoding)

Optimal_Answer  = int(CosineSimilarity.argmax())

answer_argument_description = arg_descriptions[Optimal_Answer]

answer_argument_name = ""

answer_tool = ""

for key, value in data.items():
    tmp_arg_dsc = value.get("ArgumentName")
    for argName, argDesc in tmp_arg_dsc.items():
        if (argDesc.get("Description") == answer_argument_description):
            answer_tool = key
            answer_argument_name = argName
        elif (argDesc.get("Description") == '_' and value.get("Description") == answer_argument_description):
            answer_tool = key
            answer_argument_name = key

print("Query Entered: \t\t", query)

print("Tool Name: \t\t",answer_tool)

print("Argument Name: \t\t", answer_argument_name)

print("Argument Description: \t", answer_argument_description)


Query Entered: 		 Priority List
Tool Name: 		 prioritize_objects
Argument Name: 		 objects
Argument Description: 	 A list of objects to be prioritized
